In [1]:
%pip install google-cloud-aiplatform
%pip install langchain

%pip install faiss-cpu #Facebook AI Similarity Search

Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 3.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   -------------- ------------------------- 0.8/2.1 MB 2.1 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.1 MB 1.9 MB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.1 MB 1.5 MB/s eta 0:00:01
   ----------------------------- ---------- 1.6/2.1 MB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 1.7 MB/s eta 0:00:00

   --- ------------------------------------  1/12 [PyYAML]
   ------ ---------------------------------  2/12 [orjson]
   -----------

ERROR: Invalid requirement: '#Facebook': Expected package name at the start of dependency specifier
    #Facebook
    ^


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open('devops-article.md', 'r') as file:
    text = file.read()

#chunk_size - max number of characters in a chunk
#chunk_overlap - number of characters overlapping between two adjacent chunks
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=5000, chunk_overlap=350)

docs = text_splitter.create_documents([text])

print (f"{len(docs)} docs ")


In [ ]:
from langchain.embeddings import VertexAIEmbeddings
from langchain.vectorstores import FAISS

#FAISS.from_documents - Return VectorStore initialized from documents and embeddings.
vector_store = FAISS.from_documents(docs, VertexAIEmbeddings())

question = "What are DevOps Transformation Best Practices?"

#similar_docs = vector_store.similarity_search(question, k=2)

similar_docs_with_score = vector_store.similarity_search_with_score(question, k=2)

print(similar_docs_with_score)



In [ ]:
from langchain.llms import VertexAI
from langchain.chains.question_answering import load_qa_chain

llm = VertexAI(
    model_name='text-bison@001',
    temperature=0.0,
    verbose=True,
)

#question = 'What are DevOps Transformation Best Practices?'
question = 'DevOps Maturity Signals related to Development '
#question = 'What are some of the most important DevOps metrics?'

similar_docs = vector_store.similarity_search(question, k=2)

chain = load_qa_chain(llm, chain_type='stuff')

print(chain.run(input_documents=similar_docs, question=question).strip())
